In [1]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
from astropy.io import ascii

In [2]:
def F_nu(t_p,nu_p,F_nup,nu_sed):
    '''Calculates a Synchrotron Self-Absorbed Spectrum for given input parameters.
    
    Parameters:
    t_p (days): Time of SSA peak - should be single number
    nu_p (GHz): Frequency of SSA peak - should be single number
    F_nup (mJy): Flux of SSA peak - should be single number
    nu (GHz): Frequencies at which you want the SSA spectrum to be calculated at. Likely an array.
    
    Outputs:
    Fnu (mJy): the flux of the SSA. This is an array with values associated with each value of input array nu.
    '''
    
    m = 0.88 #using stAandard model
    p = 3.0 #using standard model
    a = (2*m)+0.5
    b = (p+5-(6*m))/2
    t=t_p
    Fnu = F_nup*1.582*(t/t_p)**a*(nu_sed/nu_p)**(5/2)*(1-np.exp(-(t/t_p)**(-(a+b))*(nu_sed/nu_p)**(-(p+4)/2)))
    
    return Fnu


def SSA_props(t_p,nu_p,F_nup,D,f=0.5,alpha=1,vw=100,epsilon_b=0.1):
    '''Calculates Synchroton Self-Absorption properties for given input parameters.
    
    Inputs:
    t_p (days): Time of SSA peak - likely a single number
    nu_p (GHz): Frequency of SSA peak - likely a single number
    F_nup (mJy): Flux of SSA peak - likely a single number
    D (Mpc): distance to SN - likely a single number
    f (unitless): filling factor (fraction of emitting region). default is 0.5
    alpha (unitless): ratio charged particles to magnetic field (epsilon_e/epsilon_b). default 1
    v_w (km/s): wind speed. default = 100
    epsilon_b (unitless): fraction of shock energy into B-fields. default = 0.1
    
    Outputs:
    R (cm): radius of material 
    B : magnetic field flux
    E : intermal energy of emitting material
    v (km/s): expansion velocity of material
    M (1d-5 solar masses per year): inferred mass loss rate of progenitor'''
    

    #Radius
    R = 4.0e14*(alpha)**(-1/19)*(f/0.5)**(-1/19)*(F_nup)**(9/19)*(D)**(18/19)*(nu_p/5)**(-1)

    # Magnetic field flux
    B = 1.1*(alpha)**(-4/19)*(f/0.5)**(-4/19)*(F_nup)**(-2/19)*(D)**(-4/19)*(nu_p/5)

    #Internal energy of the emitting material
    E = (1/epsilon_b)*((B**2)/(8* 3.142))*((4*3.142*f*R**3)/3)

    #expansion velocity in km/s
    v = (R/t_p)*1.1574e-10

    #pre-explosion mass-loss in 1e-5 solar mass per year
    M = 1.0*(alpha)**(-8/19)*epsilon_b*(f)**(-8/19)*(F_nup)**(-4/19)*(D)**(-4/19)*(nu_p/5)*(t_p/10)*(vw/1000)
    
   

    return R,B,E,v,M


def taufreefree(M,R,nu):
    '''Calculates the free free optical depth for a given set of parameters.
    
    Inputs:
    M (1d-5 solar masses per year): mass loss rate
    R (cm): radius of emitting material
    nu (GHz): Frequencies at which you want tau-ff to be calculated at. Likely an array.
    
    Outputs:
    tau_ff (unitless): the free-free optical depth, calculated at same frequences as input array nu'''

    Z_ave = 5.4  # Average metallicity 1= pure H. 5.4 for a massive star
    miu = 1.9 # mean molecular weight of electrons. 1= pure H  1.9 is for a massive star.
    vw_cgs = 100 * 1e5 # assumed wind velocity in cgs (cm/s). Take this as 1000 * 10^5 for now. (i.e. 1000 km/s in cgs)
    T = 10**4 # temperature of the material absorbing in K.  10^4 is a good starting point. 
        
    M_cgs =  M * 1e-5 * 6.307e+25 #mass loss rate in cgs units
    
    tau_ff = 2.021e25*M_cgs**2*Z_ave/(miu**2*nu**(2.1)*R**3*vw_cgs**2*T**(1.35))  
    
    return tau_ff

def den(M,R):
    vw_cgs = 100 * 1e5
    M_cgs =  M * 1e-5 * 6.307e+25 #mass loss rate in cgs units 
        #density of the CSM
    density = M_cgs/(4*3.142*R**2*vw_cgs)
    
    return density
#freqs,SED = F_nu(1224.288326 , 8,  0.0272)       
#R2,B2,E2,v2,M2 = SSA_props(1224.288326 , 8,  0.0272,880) 
#tauff=taufreefree(M2,R2,freqs)
#tauff=taufreefree(4,3e14,np.array([2,5,7]))
#print(freqs)
#print(tauff)
#print(R2,B2,E2,v2,M2)

In [12]:
### Define parameters that are specific to the supernova you want to model:###
t_p = 1690.01477832512 # time of observation in days #1261.509031#
D = 1088.1 # distance to SN in Mpc
F_ul = [0.0246,0.211,0.0221] 
F_det = 0.0318
F_det_err = 0.01
nu_det = 6 #0.0404 #0.0404 # flux of upper limit in mJy
nu_ul = [10,3,15] # frequency of upper limit in GHz
file_out = 'test1.csv' #File that you will save the results in.
#plt.plot(6,0.0318, 'ro', markersize=10)
#plt.plot(10,0.0246, 'mv', markersize=10)
#plt.plot(3,0.211, 'kv', markersize=10)
#plt.plot(15,0.0221, 'gv', markersize=10)


### Define the grid of F_p and nu_p that you want to search over:###
#F_p = [0.01,0.015,0.02,0.025,0.03] #mJy
#nu = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]) # GHz

F_p = np.logspace(np.log10(0.001),np.log10(1.0),num=25) #mJy (this is an array evenly spaced in log between 0.01 annd 1)
nu_p = np.logspace(np.log10(0.05),np.log10(50),num=25) #GHz (this is an array evenly spaced in log between 0.5 annd 50)


### Define array of frequencies to calculate the SSA and SSA+FFA spectrum at ###
nu_sed = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

### Define empty arrays of the values that you want to save for each value in the grid you are searching over: ###
Fp_g = [] #peak flux
nup_g =[] #peak frequency
R_g =[] #radius
B_g = [] #Bfield
vsh_g = [] #velocity of shock
M_g = [] #mass loss rate
den_g = []
RuledOut_g = [] #flag for whether it is ruled out

######### SEARCH OVER GRID ############

for F in F_p:
    for nu in nu_p:
        #calculate SSA
        SSA_sed = F_nu(t_p,nu,F,nu_sed)
        
        #calculate Mass loss rate, Radius, velocity, etc.
        R,B,E,v,M = SSA_props(t_p,nu,F,D)
        
        #calculate the density of the CSM
        density = den(M,R)
       
        #calculate tau_ff
        tauff = taufreefree(M,R,nu_sed)
        
        #Correct SED for FFA:
        SSA_FFA_sed = SSA_sed*np.exp(-tauff)
        
        
        #Evaluate if this SED is allowed:
        RuledOut = False #Assume a default posisition that this combination is allowed.
        
        #First loop over the upper limits and see if any of them rule it out.
        for i in range(len(F_ul)):
            freq = nu_ul[i]
            flux = F_ul[i]
            #Interpolate this SSA+FFA to the frequency of your data point:
            flux_test1 = np.interp(nu_ul, nu_sed, SSA_FFA_sed)
            if np.any(flux_test1 > flux):
                RuledOut = True
                break

        #Now test against the detection as well:
        flux_test2 = np.interp(nu_det, nu_sed, SSA_FFA_sed)
        #NB: in this case 0.0318, but we should be writing this code as generally as possible.
        F_upper = F_det + F_det_err
        F_lower = F_det - F_det_err

        if (flux_test2 > F_upper):
            RuledOut = True
        if (flux_test2 < F_lower):
            RuledOut = True

        
        # Append the values from this loop into the arrays that we defined above:
        Fp_g.append(F) 
        nup_g.append(nu) 
        R_g.append(R) 
        B_g.append(B) 
        vsh_g.append(v) 
        M_g.append(M)
        den_g.append(density)
        RuledOut_g.append(RuledOut) 
 
        #It will now go on to the next set in the loop:
    
# Now it has gone over the whole loop. So write out the results into a data file that you can use later:
data = [Fp_g,nup_g,R_g,B_g,vsh_g,M_g,den_g,RuledOut_g]
names = ['F_peak','nu_peak','Radius','Bfield','v_shock','Mdot','den','RuledOut']
ascii.write(data,file_out,names=names,overwrite=True,format='csv')


In [13]:
file_in = 'test1.csv'
data_plot = ascii.read(file_in)

gammabeta = data_plot['v_shock']*(1/3e5)



plt.figure()

for i in range(len(data_plot)):
    if data_plot['RuledOut'][i] == 'True':
        plt.plot(data_plot['Mdot'][i],gammabeta[i],'ro',alpha=0.5)
    else:
        plt.plot(data_plot['Mdot'][i],gammabeta[i],'ko', label ='PS11aop8')#,alpha=0.2)

plt.yscale('log')
plt.xscale('log')
plt.xlabel(r'Mass Loss Rate ($10^{-5}$ Msun/year)')
plt.ylabel('Gamma*Beta')
plt.title('Parameter space for mass-loss rates allowed for PS11aop')
#plt.show()
#plt.xlim(0.01,0.1)
#plt.ylim(0.1,10)
plt.savefig('mp.pdf')